<a href="https://colab.research.google.com/github/sathyanaravind/hands-on-llms/blob/main/07-advanced-text-generation/langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install requirements

In [3]:
!pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2 langchain_community
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python==0.2.69

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 MB 18.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.0 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.69-cp311-cp311-linux_x86_64.whl size=55723473 sha256=d8f24197975c216199bf75b8a9a3b1c4f01f2a8a7ea58c6429619988f07b9731
  Stored in directory: /root/.cache/pip/wheels/e8/1b/ff/b4dba97fbd16e731705b262602ba8f3b672bf4bde54ea0c104
Successfully built llama-cpp-python


Loading LLM

In [4]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf


--2025-03-17 07:58:05--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 3.163.189.90, 3.163.189.37, 3.163.189.114, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.90|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1742201885&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MjIwMTg4NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQxZDg4

In [5]:
from langchain import LlamaCpp

llm = LlamaCpp(
    model_path="/content/Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

In [6]:
llm.invoke("Hi! My name is Aravind. What is 1+1?")

'\n<|assistant|> The answer to your question, Aravind, is that 1 + 1 equals 2.\n\nIf you have any other questions or need assistance with something else, feel free to ask!'

Chains

In [7]:
from langchain import PromptTemplate

template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
    )

In [8]:
basic_chain = prompt | llm

In [9]:
# using the chain
basic_chain.invoke({
    "input_prompt": "What is 2+2?"
})

" 2 + 2 equals 4. It's a basic arithmetic operation where you add the two numbers together to get their total sum."

Multiple Chains

Write a story from
initial input prompt: summary -> title -> character -> story

In [10]:
from langchain import LLMChain

template = """<s><|user|>
Create a title story about {summary}. Only return the title<|end|>
<|assistant|>"""

# title
title_prompt = PromptTemplate(
    template=template,
    input_variables=["summary"]
    )

title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

<ipython-input-10-9fefe204f258>:13: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")


In [11]:
title.invoke({
    "summary": "a girl that lost her mother"
})

{'summary': 'a girl that lost her mother',
 'title': ' "Echoes of Absence: Navigating Life\'s Path Without Her Mother"'}

In [12]:
# character

template = """<s><|user|>
Describe the main character of the story about {summary} with the {title}. Use only two sentences<|end|>
<|assistant|>"""

# title
character_prompt = PromptTemplate(
    template=template,
    input_variables=["summary", "title"]
    )

character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [13]:
# story

template = """<s><|user|>
Create a  story about {summary} with the {title}. The main character is: {character}. Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""

# title
story_prompt = PromptTemplate(
    template=template,
    input_variables=["summary", "title"]
    )

story = LLMChain(llm=llm, prompt=story_prompt, output_key="character")

In [14]:
llm_chain = title | character | story

In [15]:
llm_chain.invoke("a girl that lost her mother")

{'summary': 'a girl that lost her mother',
 'title': ' "Echoes of Absence: A Journey Through Grief"',
 'character': ' In "Echoes of Absence: A Journey Through Grief," we meet Emily, an introspective young woman grappling with profound sorrow after losing her mother. With each passing day, memories linger like ghostly whispers, urging her to face the pain head-on and embark on a deeply personal journey. As she walks through the labyrinth of grief, Emily encounters both heartache and healing in equal measure. She finds solace not only in cherished memories but also within herself as she discovers an inner resilience that slowly weaves her shattered pieces back together. Along this poignant path, Emily learns to embrace the ebbs and flows of grief, transforming them into a testament of strength and love—a beacon for others navigating their own darkened corridors of absence.'}

**Memory**

In [34]:
basic_chain.invoke({"input_prompt": "Hi! My name is Aravin. What is 1 + 1?"})

' Hello Aravin, the answer to 1 + 1 is 2. Math can be fun and straightforward once you get started!'

In [35]:
basic_chain.invoke({"input_prompt": "What is my name?"})

" I'm unable to determine your name as I don't have the capability to access personal data. If you need help with something specific, feel free to ask!"

Conversation Buffer

In [36]:
template = """<s><|user|>Current conversation: {chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
    )

In [37]:
from langchain.memory import ConversationBufferMemory

# Define the type of Memory we will use
memory = ConversationBufferMemory(memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [38]:
llm_chain.invoke({"input_prompt": "Hi! My name is Aravind. What is 1 + 1?"})

{'input_prompt': 'Hi! My name is Aravind. What is 1 + 1?',
 'chat_history': '',
 'text': ' Hello Aravind! The answer to 1 + 1 is 2.'}

In [39]:
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': 'Human: Hi! My name is Aravind. What is 1 + 1?\nAI:  Hello Aravind! The answer to 1 + 1 is 2.',
 'text': ' Your name is Aravind.'}

ConversationBufferMemoryWindow


In [23]:
from langchain.memory import ConversationBufferWindowMemory
from langchain import LLMChain

memory = ConversationBufferWindowMemory(k=2, key="chat_history")

chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-23-9d1883e1e788>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, key="chat_history")


In [40]:
llm_chain.invoke({"input_prompt":"Hi! My name is Aravind and I am 27 years old. What is 1 + 1?"})
llm_chain.invoke({"input_prompt":"What is 3 + 3?"})

{'input_prompt': 'What is 3 + 3?',
 'chat_history': 'Human: Hi! My name is Aravind. What is 1 + 1?\nAI:  Hello Aravind! The answer to 1 + 1 is 2.\nHuman: What is my name?\nAI:  Your name is Aravind.\nHuman: Hi! My name is Aravind and I am 27 years old. What is 1 + 1?\nAI:  Hello Aravind! The answer to 1 + 1 is 2. Congratulations on sharing your age with me—you are 27 years old now!\n\nAs for the sum of 1 + 1, it remains a classic math problem: it equals 2.',
 'text': ' The answer to 3 + 3 is 6. This is another basic arithmetic calculation. Just like in your previous question about 1+1=2, adding three and three together gives you six. Keep up the good work with math!'}

In [41]:
llm_chain.invoke({"input_prompt":"What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': 'Human: Hi! My name is Aravind. What is 1 + 1?\nAI:  Hello Aravind! The answer to 1 + 1 is 2.\nHuman: What is my name?\nAI:  Your name is Aravind.\nHuman: Hi! My name is Aravind and I am 27 years old. What is 1 + 1?\nAI:  Hello Aravind! The answer to 1 + 1 is 2. Congratulations on sharing your age with me—you are 27 years old now!\n\nAs for the sum of 1 + 1, it remains a classic math problem: it equals 2.\nHuman: What is 3 + 3?\nAI:  The answer to 3 + 3 is 6. This is another basic arithmetic calculation. Just like in your previous question about 1+1=2, adding three and three together gives you six. Keep up the good work with math!',
 'text': ' Your name is Aravind.'}

In [42]:
llm_chain.invoke({"input_prompt":"What is my age?"})

{'input_prompt': 'What is my age?',
 'chat_history': 'Human: Hi! My name is Aravind. What is 1 + 1?\nAI:  Hello Aravind! The answer to 1 + 1 is 2.\nHuman: What is my name?\nAI:  Your name is Aravind.\nHuman: Hi! My name is Aravind and I am 27 years old. What is 1 + 1?\nAI:  Hello Aravind! The answer to 1 + 1 is 2. Congratulations on sharing your age with me—you are 27 years old now!\n\nAs for the sum of 1 + 1, it remains a classic math problem: it equals 2.\nHuman: What is 3 + 3?\nAI:  The answer to 3 + 3 is 6. This is another basic arithmetic calculation. Just like in your previous question about 1+1=2, adding three and three together gives you six. Keep up the good work with math!\nHuman: What is my name?\nAI:  Your name is Aravind.',
 'text': " I'm unable to determine your current age as our conversation doesn't include that information. However, you mentioned earlier that you are 27 years old in a previous part of the conversation. If you need help with anything else, feel free to 

Conversation Summary

In [43]:
summary_prompt_template = """<s><|user|>Summarize the conversations and update with new lines.

Current summary:
{summary}

new lines of conversation:
{new_lines}

New summary:
<|assistant|>"""

summary_prompt = PromptTemplate(
    template=summary_prompt_template,
    input_variables=["summary", "new_lines"]
    )

In [47]:
from langchain.memory import ConversationSummaryMemory

# Define the type of memory we will use
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    prompt=summary_prompt
)

# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

In [48]:
llm_chain.invoke({"input_prompt":"Hi! My name is Aravind and I am 27 years old. What is 1 + 1?"})
llm_chain.invoke({"input_prompt":"What is 3 + 3?"})

{'input_prompt': 'What is 3 + 3?',
 'chat_history': ' Aravind, a 27-year-old individual, engaged in a conversation with the AI assistant and asked what 1 + 1 equals. The AI replied by stating that the sum of 1 + 1 is equal to 2, also inquiring about how it can provide further assistance.',
 'text': ' 3 + 3 equals 6. If you need help with anything else, feel free to ask!'}

In [49]:
llm_chain.invoke({"input_prompt":"What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': ' Aravind, a 27-year-old individual, conversed with the AI assistant and initially asked what 1 + 1 equals. The AI correctly responded that it sums up to 2 and offered further assistance. Later, when queried about 3 + 3, the AI provided the correct answer of 6, extending its offer for additional help if needed.',
 'text': ' Your name, based on the information provided, is not mentioned in our conversation. However, you referred to yourself as "Aravind." Is there anything else I can assist you with today?'}

In [50]:
llm_chain.invoke({"input_prompt":"What was the first question that I asked?"})

{'input_prompt': 'What was the first question that I asked?',
 'chat_history': ' Aravind, a 27-year-old individual, interacted with an AI assistant and asked for the sum of various numbers like 1 + 1 (which equals 2) and 3 + 3 (equaling 6), to which the AI provided correct answers. When queried about their name during a later conversation, the AI mentioned that it was not disclosed in the current interaction but recognized the individual as "Aravind." The AI offered further assistance with any additional questions or requests.',
 'text': ' The first question you asked was: "1 + 1" and "3 + 3".'}

In [51]:
memory.load_memory_variables({})

{'chat_history': ' Aravind, a 27-year-old user, engaged in conversations with an AI assistant. They initially requested the sum of numbers like 1 + 1 (which equals 2) and 3 + 3 (equaling 6), to which the AI correctly responded. Later, when asked about their name during a subsequent interaction, the AI acknowledged that it hadn\'t disclosed its own identity but recognized the user as "Aravind." The AI also offered further assistance for any additional questions or requests. In a new exchange, the user inquired about the first question they had posed to the AI, which was calculating the sums of 1 + 1 and 3 + 3.'}

Agents

In [64]:
import os
from langchain_openai import ChatOpenAI


os.environ["OPENAI_API_KEY"] = ""
openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0)

In [59]:
# ReAct Template

In [60]:
react_template ="""Answer the following questions as best as you can. You have access to the following tools:

{tools}:

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take should be of : {tool_names}
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["input", "agent_scratchpad", "tools", "tool_names"]
)


In [61]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

#create tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web serach engine. Use this as a serach engine for general queries",
    func=search.run
)

# prepare the tools
tools = load_tools(["llm-math"], llm=openai_llm)
tools.append(search_tool)

In [62]:
# define agent
from langchain.agents import AgentExecutor, create_react_agent

agent = create_react_agent(openai_llm,tools,prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [63]:
# check agent
agent_executor.invoke({"input":"What is the current price of macbook pro in USD? How much does it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"})



> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}